# Parse KosciolekAndJones raw PDBs

In [1]:
# reload modules before executing code in order to make development and debugging easier
%load_ext autoreload
%autoreload 2

In [2]:
# this jupyter notebook is running inside of the "notebooks" directory
# for relative paths to work properly, we need to set the current working directory to the root of the project
# for imports to work properly, we need to add the code folder to the system path
import os
from os.path import abspath, join, isdir, basename, isfile
import sys
if not isdir("notebooks"):
    # if there's a "notebooks" directory in the cwd, we've already set the cwd so no need to do it again
    os.chdir("..")
module_path = abspath("code")
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import collections
import itertools 
import shutil
import textwrap

from Bio import SeqIO
from Bio.SeqIO.PdbIO import AtomIterator
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
from Bio.Seq import Seq 
from Bio.SeqUtils import seq1
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
from biopandas.pdb import PandasPdb

import pandas as pd
import numpy as np

In [4]:
import warnings

# Grab the raw PDBs

In [5]:
pdb_dir = "pdb_files/KosciolekAndJones/pdbs_raw"
pdb_fns = [join(pdb_dir, x) for x in os.listdir(pdb_dir) if x.endswith(".pdb")]
pdb_ids = [basename(x)[:-4] for x in pdb_fns]

# Identify missing residues with the SEQRES method
For each PDB file, read the sequence from (1) SEQRES records and (2) ATOM records. If sequences match, they *should be* good to go (no missing residues). However, some of them might have **modified** residues. Biopython parses modified residues as regular residues, so it's impossible to tell which residues are modified based on the sequence returned by Biopython. In the PDB file, the modified residues show up as modified in the MODRES, SEQRES, and ATOM records. The modified residues are usually listed as HETATM instead of ATOM.

Another way to check for missing residues could be to look at the residue numbers and make sure they are sequential without any gaps. However, this fails if the missing residues are at the start or end of the sequence. So prefer the SEQRES method.

In [10]:
# list of sequences from SEQRES
seqres_seqs = []
for pdb_fn in pdb_fns:
    found_chain_A = False
    for record in SeqIO.parse(pdb_fn, "pdb-seqres"):
        if record.annotations["chain"] == "A":
            found_chain_A = True
            seqres_seqs.append(str(record.seq))
    if not found_chain_A:
        print("no chain A for {}".format(basename(pdb_fn)))

# list of sequences from ATOM records
atom_seqs = []
with warnings.catch_warnings(record=True):
    # suppress biopython warnings because the atom records are missing residues
    for pdb_fn in pdb_fns:
        found_chain_A = False
        for record in SeqIO.parse(pdb_fn, "pdb-atom"):
            if record.annotations["chain"] == "A":
                found_chain_A = True
                atom_seqs.append(str(record.seq))
        if not found_chain_A:
            print("no chain A for {}".format(basename(pdb_fn)))

In [7]:
# find the PDBs where the seqres seqs and the atom seqs are equivalent
# these *should be* ready-to-go because all residues that should be present 
# according to seqres are in fact present in the atoms
equiv = []
non_equiv = []
for pdb_fn, seqres_seq, atom_seq in zip(pdb_fns, seqres_seqs, atom_seqs):
    if seqres_seq == atom_seq:
        equiv.append(pdb_fn)
    else:
        non_equiv.append(pdb_fn)

missing_residues = [basename(x)[:-4] for x in non_equiv]
not_missing_residues = [basename(x)[:-4] for x in equiv]

In [8]:
print("Not missing residues: {}".format(len(not_missing_residues)))
print("Missing residues: {}".format(len(missing_residues)))

Not missing residues: 50
Missing residues: 100


## Not actually missing residues

Note! There are some PDBs that were identified as having missing residues in the above section, but they don't actually have missing residues. Instead, they have extra residues being read in from the ATOM seqs due to weirdness with the ATOM seqs. These PDBs are '1bsg', '1dix', '1i4j', '1ek0', '5ptp'. They get processed successfully with Rosetta's clean_pdb.py, so we can just use the raw PDBs for them (or the chain A-only "clean" PDBs). 

This was discovered after ready_set_1.txt was created (below), so they are going to be processed with the second round of PDBs (along with ones that got remodeled due to having missing residues).

In [9]:
idx = pdb_ids.index("1gz2")
print("seqres == atom:", seqres_seqs[idx] == atom_seqs[idx])
print("seqres len: {}, atom len: {}".format(len(seqres_seqs[idx]), len(atom_seqs[idx])))
print("seqres seq:")
print("\n".join(textwrap.wrap(seqres_seqs[idx])))
print("atom seq:")
print("\n".join(textwrap.wrap(atom_seqs[idx])))

seqres == atom: False
seqres len: 142, atom len: 139
seqres seq:
DPDGCGPGWVPTPGGCLGFFSRELSWSRAESFCRRWGPGSHLAAVRSAAELRLLAELLNASRGGDGSGEG
ADGRVWIGLHRPAGSRSWRWSDGTAPRFASWHRTAKARRGGRCAALRDEEAFTSWAARPCTERNAFVCKA
AA
atom seq:
GCGPGWVPTPGGCLGFFSRELSWSRAESFCRRWGPGSHLAAVRSAAELRLLAELLNASRGGDGSGEGADG
RVWIGLHRPAGSRSWRWSDGTAPRFASWHRTAKARRGGRCAALRDEEAFTSWAARPCTERNAFVCKAAA


## Actually missing residues

There are 2 PDB files that are actually missing residues: 1c52 and 1fx2. They have residues with zero occupancy that are removed by clean_pdb.py.

# Identify modified residues with MODRES records
PandasPDB makes it easy to parse other records, such as MODRES.

In [18]:
ppdbs = [PandasPdb().read_pdb(pdb_fn) for pdb_fn in pdb_fns]

In [19]:
modified_residues = []
for pdb_id, ppdb in zip(pdb_ids, ppdbs):
    others = ppdb.df["OTHERS"]
    if "MODRES" in others.record_name.unique():
        # modified residue only a concern if it's a different amino acid code
        # i think glycosylation site and disulfide bridge aren't a problem
        # but just in case, should check with Phil
        mr = others[others.record_name == "MODRES"]
        concerning_modres = mr["entry"].apply(lambda x: x[6:9] != x[18:21]).any()
        if concerning_modres:
            modified_residues.append(pdb_id)
#         print(pdb_id)
#         print(others[others.record_name == "MODRES"])

In [20]:
print("Modified residues: {}".format(len(modified_residues)))

Modified residues: 21


# Figure out which PDBs are ready to go
And which need more work.

In [21]:
missing_only = set(missing_residues) - set(modified_residues)
missing_and_modified = set(missing_residues).intersection(set(modified_residues))
modified_only = set(modified_residues) - set(missing_residues)
ready_set_1 = set(not_missing_residues) - set(modified_residues)

In [22]:
print("PDBs with only missing residues, no modified residues ({})\n{}".format(len(missing_only), " ".join(missing_only)))

PDBs with only missing residues, no modified residues (83)
1cxy 1jkx 2tps 1chd 1guu 1dmg 1d1q 1nrv 1g2r 1cc8 1hxn 1fl0 1vmb 1a3a 1fqt 1xdz 1jo0 1mug 1dqg 3dqg 1hh8 1bkr 1hdo 2arc 1kqr 1roa 1k6k 1bsg 1xff 1ihz 1kid 1lo7 1aap 1tzv 1dix 3bor 1m8a 1jvw 1xkr 1im5 1d4o 1qjp 1beh 1tqh 1ctf 1ryb 1ku3 1eaz 1r26 1jfu 1vjk 1m4j 1smx 1atl 1o1z 1i1n 1jl1 1i4j 1fvk 1wjx 1d0q 1iib 1jos 1vp6 1avs 2vxn 1hfc 1atz 1jbk 2cua 1vfy 1ktg 1rw7 1gmi 1tif 1cke 1beb 1p90 1jyh 1j3a 1i1j 1h98 1f6b


In [23]:
print("PDBs with missing residues and modified residues ({})\n{}".format(len(missing_and_modified), " ".join(missing_and_modified)))

PDBs with missing residues and modified residues (17)
1svy 1fvg 1kq6 1pko 1ek0 1ne2 1h4x 1lm4 1w0h 1gz2 1wkc 1kw4 1vhu 1dbx 1ny1 1lpy 5ptp


In [24]:
print("PDBs with no missing residues, only modified residues ({})\n{}".format(len(modified_only), " ".join(modified_only)))

PDBs with no missing residues, only modified residues (4)
1rw1 1gmx 1jbe 1k7j


In [25]:
print("PDBs with no missing residues and no modified residues ({})\n{}".format(len(ready_set_1), " ".join(ready_set_1)))

PDBs with no missing residues and no modified residues (46)
1a70 1pch 1c9o 1gbs 1k7c 1a6m 1dsx 1ej8 1ej0 1ql0 1cjw 1jwq 1dlw 1c52 1g9o 1i5g 1brf 1jo8 1h2e 1fna 1tqg 1gzc 1fk5 2mhr 1c44 1whi 1i58 1t8k 1aoe 1aba 2hs1 1i71 1fx2 1bdo 1iwd 1jfx 1nb9 1ag6 1nps 1h0p 1mk0 2phy 1czn 1qf9 1fcy 1htw


# Create list of ready-to-go PDBs (ready_set_1)
This is ready_set_1.txt. PDB files from the list of 150 that are ready for the prepare pipeline without having to deal with missing or modified residues. 

Note this list is missing some PDBs that were originally thought to have missing residues, but do not (see section 3.1). Additionally, some PDBs with modified residues are also ready-to-go in that Rosetta's clean_pdb will fill in modified residues with canonical amino acids. However, these PDBs with modified residues were not included in this list.

Note this list includes 2 PDBs that have residues with zero occupancy and are thus filtered out with clean_pdb.py (leaving missing residues). These were remodeled and need to be re-run through energize.

In [142]:
with open("pdb_files/KosciolekAndJones/ready_set_1.txt", "w") as f:
    for pdb_id in sorted(ready_set_1):
        f.write(f"pdb_files/KosciolekAndJones/pdbs_chains/{pdb_id}_A.pdb\n")

## Copy prepared PDB files from ready_set_1 to main prepared directory
In between this step and previous step, the pdb files from ready_set_1.txt were run through the prepare pipeline.

In [63]:
with open("pdb_files/KosciolekAndJones/ready_set_1.txt", "r") as f:
    rs1_pdb_ids = [basename(x.strip())[:-4] for x in f.readlines()]

# copy from prepare_outputs directory to prepared_pdb_files
prep_outputs_dir = "output/prepare_outputs"
outputs_mapping = {}
for d in os.listdir(prep_outputs_dir):
    outputs_mapping[d[:6]] = join(prep_outputs_dir, d, "{}_p.pdb".format(d[:6]))

final_output_dir = "pdb_files/prepared_pdb_files"
for pdb_id in rs1_pdb_ids:
    shutil.copy(outputs_mapping[pdb_id], final_output_dir)
#     print(outputs_mapping[pdb_id])

# Create list of ready-to-go PDBs (ready_set_2)

This is ready_set_2.txt. PDB files that have modified residues that are still able to be processed by clean_pdb and have the modified residues replaced with canonical residues. Also PDB files that were thought to have missing residues, but actually don't. 

In [149]:
ready_set_2 = ["1rw1", "1gmx", "1k7j", "1bsg", "1dix", "1i4j", "1ek0", "5ptp"]
with open("pdb_files/KosciolekAndJones/ready_set_2.txt", "w") as f:
    for pdb_id in sorted(ready_set_2):
        # note we are processing them from the RAW pdb's instead of pdbs_chains this time. 
        # clean_pdb will take care of selecting chain A (as long as we specify it when we call the prepare pipeline)
        # this is taken care of in prepare_2.sh
        # could probably just ues pdbs_chains like last time, but I tested it pdbs_raw, so keep it that way for now
        f.write(f"pdb_files/KosciolekAndJones/pdbs_raw/{pdb_id}.pdb\n")

## Copy prepared PDB files from ready_set_2 to main prepared directory

In [168]:
with open("pdb_files/KosciolekAndJones/ready_set_2.txt", "r") as f:
    rs2_pdb_ids = [basename(x.strip())[:-4] for x in f.readlines()]

# copy from HTCondor prepare_outputs directory to prepared_pdb_files
prep_outputs_dir = "output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs"
outputs_mapping = {}
for d in os.listdir(prep_outputs_dir):
    outputs_mapping[d[:4]] = join(prep_outputs_dir, d, "{}_p.pdb".format(d[:4]))

final_output_dir = "pdb_files/prepared_pdb_files"
for pdb_id in rs2_pdb_ids:
    shutil.copy(outputs_mapping[pdb_id], final_output_dir)
    print(outputs_mapping[pdb_id])

output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1bsg_2022-02-10_18-39-10/1bsg_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1dix_2022-02-10_18-39-19/1dix_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1ek0_2022-02-10_18-39-12/1ek0_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1gmx_2022-02-10_18-39-25/1gmx_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1i4j_2022-02-10_18-39-13/1i4j_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1k7j_2022-02-10_18-39-14/1k7j_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/1rw1_2022-02-10_18-39-11/1rw1_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-29-37_rs2/output/prepare_outputs/5ptp_2022-02-10_18-39-08/5ptp_p.pdb


# Create list of ready-to-go PDBs (ready_set_3) 
These are from loop modeling. Note, I originally filtered out 1c52 and 1fx2 but these actually DO need to be taken from remodel rather than raw. So, I manually ran them through prepare afterward, but then updated this code to not filter them out, so in case this needs to be run in the future, it can just be run from here.

In [163]:
checks_fn = "pdb_files/KosciolekAndJones/loop_modeling/data/remodel_checks.csv"
checks = pd.read_csv(checks_fn)
checks.columns = ["code", "passed"]
checks["passed"] = checks["passed"].apply(lambda x: True if x.strip() == "True" else False)

ready_set_3 = set(checks[checks["passed"]]["code"])
# note there are 2 PDBs that were remodeled but don't appear to be missing any residues (1c52 and 1fx2)
# make sure to remove these from ready_set_3 as we don't need to re-run them
# to accomplish this we just remove ready_set_1 and ready_set_2 to be sure we aren't running duplicates
# EDIT: Nevermind, these DO need to be included.
# ready_set_3 = ready_set_3 - ready_set_1 - set(ready_set_2)

with open("pdb_files/KosciolekAndJones/ready_set_3.txt", "w") as f:
    for pdb_id in sorted(ready_set_3):
        # note these are coming from the pdbs_remodel directory (from root KosciolekAndJones folder, not loop_modeling)
        # these PDB files are the same as the ones from the loop modeling but have been renamed _remod
        f.write(f"pdb_files/KosciolekAndJones/pdbs_remodel/{pdb_id}_remod.pdb\n")

## Copy prepared PDB files from ready_set_3 to main prepared directory

In [172]:
with open("pdb_files/KosciolekAndJones/ready_set_3.txt", "r") as f:
    rs3_pdb_ids = [basename(x.strip())[:-4] for x in f.readlines()]

# copy from HTCondor prepare_outputs directory to prepared_pdb_files
prep_outputs_dir = "output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs"
outputs_mapping = {}
for d in os.listdir(prep_outputs_dir):
    outputs_mapping[d[:10]] = join(prep_outputs_dir, d, "{}_p.pdb".format(d[:10]))

# print(outputs_mapping)
final_output_dir = "pdb_files/prepared_pdb_files"
for pdb_id in rs3_pdb_ids:
    shutil.copy(outputs_mapping[pdb_id], final_output_dir)
    print(outputs_mapping[pdb_id])

output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1a3a_remod_2022-02-10_19-12-34/1a3a_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1aap_remod_2022-02-10_19-42-25/1aap_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1atl_remod_2022-02-10_19-41-45/1atl_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1atz_remod_2022-02-10_19-49-05/1atz_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1avs_remod_2022-02-10_19-48-49/1avs_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1beb_remod_2022-02-10_19-50-57/1beb_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/1beh_remod_2022-02-10_19-51-23/1beh_remod_p.pdb
output/htcondor_runs/condor_prepare_2022-02-10_12-58-47_rs3/output/prepare_outputs/

In [ ]:
# 

# Examine outputs from loop modeling (testing stuff out)
PDBs that were not "ready to go" were run through the loop modeling protocol to fill in missing residues.

In [136]:
checks_fn = "pdb_files/KosciolekAndJones/loop_modeling/data/remodel_checks.csv"

In [137]:
checks = pd.read_csv(checks_fn)
checks.columns = ["code", "passed"]
checks["passed"] = checks["passed"].apply(lambda x: True if x.strip() == "True" else False)

PDBs that were remodeled but don't have missing residues

In [152]:
set(checks[checks["passed"]]["code"]) - missing_only - missing_and_modified

{'1c52', '1fx2'}

PDBs that have missing residues but weren't remodeled

In [78]:
missing_only - set(checks["code"])

{'1bsg', '1dix', '1i4j'}

In [79]:
missing_and_modified - set(checks["code"])

{'1ek0', '5ptp'}

PDBs that were unsuccessfully remodeled

In [139]:
checks[checks["passed"] == True]

,code,passed
0,1a3a,True
1,1aap,True
2,1atl,True
3,1atz,True
4,1avs,True
...,...,...
93,2cua,True
94,2tps,True
95,2vxn,True
96,3bor,True


In [174]:
checks[checks["passed"] == False]

,code,passed
30,1gz2,False
33,1hdo,False
38,1i1n,False
42,1j3a,False
43,1jbe,False
59,1lm4,False


In [146]:
manual_process = ["1rw1", "1gmx", "1k7j", "1bsg", "1dix", "1i4j", "1ek0", "5ptp"]
set(pdb_ids) - set(checks[checks["passed"]]["code"]) - ready_set_1 - set(manual_process)

{'1gz2', '1hdo', '1i1n', '1j3a', '1jbe', '1lm4'}

In [173]:
set(pdb_ids) - set(ready_set_1) - set(ready_set_2) - set(ready_set_3)

{'1gz2', '1hdo', '1i1n', '1j3a', '1jbe', '1lm4'}